In [1]:
import pandas as pd
import numpy as np

# !pip install deepctr[gpu]

# 导入deepctr
from deepctr.models import DIN,DIEN,BST
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
import tensorflow as tf

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

generate='./data/generate/'
path='./data/rec_data/'

2022-05-15 00:32:08.899897: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
user_seq=pd.read_csv(generate+'user_seq_raw.csv',usecols=['suv','itemId'])
train=pd.read_csv(path+'train-dataset.csv')
test=pd.read_csv(path+'test-dataset.csv')
test.rename(columns={'testSampleId':'sampleId'},inplace=True)
category=pd.read_csv(generate+'category.csv')
category.rename(columns={"id":"itemId"},inplace=True)
train=pd.concat([train,test])
del test
train=pd.concat([train,user_seq])
train=pd.merge(train,category,how='left',on='itemId')

KeyboardInterrupt: 

In [ ]:
# 数据处理sparse、dense
for feat in ['itemId']:
    lbe = LabelEncoder()
    train[feat] = lbe.fit_transform(train[feat])

# train[['itemId']]+=1
# train[['category']]+=1
    
ygq=train.copy()
user_seq=train[train['sampleId'].isna()]
user_seq=user_seq[['suv','itemId','category']]
train=train[train['sampleId'].notna()]

In [ ]:
# 造点统计特征
train['pvid_count'] = train.groupby('pvId')['itemId'].transform('count')
train['pvid_item_nunique'] = train.groupby('pvId')['itemId'].transform('nunique')
train['pvid_suv_nunique'] = train.groupby('pvId')['suv'].transform('nunique')
train['item_count'] = train.groupby('itemId')['itemId'].transform('count')
train['item_suv_nunique'] = train.groupby('pvId')['suv'].transform('nunique')

In [ ]:
# train=train[train['userSeq'].notna()]
del train['userSeq']
del train['logTs']

In [ ]:
hist_click=user_seq[['suv','itemId','category']].groupby(['suv']).agg({list}).reset_index()
his_behavior_df=pd.DataFrame()
his_behavior_df['suv']=hist_click['suv']
his_behavior_df['hist_itemId']=hist_click['itemId']
his_behavior_df['hist_category']=hist_click['category']
del hist_click
del user_seq

In [ ]:
data=pd.merge(train,his_behavior_df,how='left',on='suv')

In [ ]:
tmps=data['hist_itemId'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
data['hist_itemId']=tmps

tmps=data['hist_category'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
data['hist_category']=tmps

# data['length'].fillna(0,inplace=True)

del data['sampleId']

In [ ]:
# 把特征分开
sparse_fea = ['suv', 'pvId','itemId', 'operator','category',
              'browserType', 'deviceType', 'osType', 'province', 'city']

behavior_fea = ['itemId','category']

hist_behavior_fea = ['hist_itemId','hist_category']

dense_fea = ['pvid_count', 'pvid_item_nunique', 'pvid_suv_nunique', 'item_count', 'item_suv_nunique']

In [ ]:
sparse_fea_tmp = ['suv', 'pvId', 'operator',
              'browserType', 'deviceType', 'osType', 'province', 'city']

# 数据处理sparse、dense
for feat in sparse_fea_tmp:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

data[dense_fea] = data[dense_fea].fillna(0, )
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_fea] = mms.fit_transform(data[dense_fea])

In [ ]:
data.to_feather(generate+'DIN_data.feather')
ygq.to_feather(generate+'DIN_data_ygq.feather')

============================================================================================================

## 上为数据处理，下为输入模型

In [1]:
import pandas as pd
import numpy as np

# !pip install deepctr[gpu]

# 导入deepctr
from deepctr.models import DIN,DIEN,BST
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
import tensorflow as tf

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

generate='./data/generate/'
path='./data/rec_data/'

2022-05-16 22:57:37.135828: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# 把特征分开
sparse_fea = ['suv', 'pvId','itemId', 'operator','category','category10','category19',
              'browserType', 'deviceType', 'osType', 'province', 'city']

behavior_fea = ['itemId','category','category10','category19']

hist_behavior_fea = ['hist_itemId','hist_category','hist_category10','hist_category19']

dense_fea = ['pvid_count', 'pvid_item_nunique', 'pvid_suv_nunique', 'item_count', 'item_suv_nunique']

In [3]:
data=pd.read_feather(generate+'DIN_data_2.feather')
ygq=pd.read_feather(generate+'DIN_data_ygq_2.feather')

In [4]:
# 数据准备函数
def get_din_feats_columns(df, dense_fea, sparse_fea, behavior_fea, his_behavior_fea, emb_dim=32, max_len=32,ygq=ygq):
    sparse_feature_columns = [SparseFeat(feat, vocabulary_size=df[feat].nunique() + 1, embedding_dim=emb_dim) for feat in sparse_fea]
    dense_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_fea]
    var_feature_columns = [VarLenSparseFeat(SparseFeat(feat, vocabulary_size=ygq[feat.split('_')[1]].nunique() + 2,
                        embedding_dim=emb_dim, embedding_name=feat.split('_')[1]), maxlen=max_len) for feat in hist_behavior_fea]
    
    dnn_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns
    
    # 建立x, x是一个字典的形式
    x = {}
    for name in get_feature_names(dnn_feature_columns):
        if name in his_behavior_fea:
            # 这是历史行为序列
            his_list = [l for l in df[name]]
            x[name] = pad_sequences(his_list, maxlen=max_len, padding='post')      # 二维数组
        else:
            x[name] = df[name].values
    
    return x, dnn_feature_columns


In [5]:
train=data[data['label'].notna()]
test=data[data['label'].isna()]
del data
del ygq
test.head(20)

,label,pvId,suv,itemId,operator,browserType,deviceType,osType,province,city,...,category19,pvid_count,pvid_item_nunique,pvid_suv_nunique,item_count,item_suv_nunique,hist_itemId,hist_category,hist_category10,hist_category19
4233804,NaN,5,319111,89719,258,26,0,8,32,186,...,11,0.041667,0.045016,0.0,0.052617,0.0,[0],[0],[0],[0]
4233805,NaN,5,319111,96838,258,26,0,8,32,186,...,12,0.041667,0.045016,0.0,0.353261,0.0,[0],[0],[0],[0]
4233806,NaN,5,319111,89846,258,26,0,8,32,186,...,10,0.041667,0.045016,0.0,1.000000,0.0,[0],[0],[0],[0]
4233807,NaN,5,319111,97643,258,26,0,8,32,186,...,5,0.041667,0.045016,0.0,0.142227,0.0,[0],[0],[0],[0]
4233808,NaN,5,319111,84057,258,26,0,8,32,186,...,1,0.041667,0.045016,0.0,0.593296,0.0,[0],[0],[0],[0]
4233809,NaN,5,319111,76838,258,26,0,8,32,186,...,10,0.041667,0.045016,0.0,0.227692,0.0,[0],[0],[0],[0]
4233810,NaN,5,319111,97232,258,26,0,8,32,186,...,10,0.041667,0.045016,0.0,0.010952,0.0,[0],[0],[0],[0]
4233811,NaN,5,319111,87006,258,26,0,8,32,186,...,11,0.041667,0.045016,0.0,0.133669,0.0,[0],[0],[0],[0]
4233812,NaN,5,319111,90928,258,26,0,8,32,186,...,18,0.041667,0.045016,0.0,0.289557,0.0,[0],[0],[0],[0]
4233813,NaN,5,319111,93331,258,26,0,8,32,186,...,12,0.041667,0.045016,0.0,0.372753,0.0,[0],[0],[0],[0]


In [6]:
def get_kfold_users(trn_df, n=5):
    user_ids = trn_df['suv'].unique()
    user_set = [user_ids[i::n] for i in range(n)]
    return user_set

k_fold = 5
trn_df = train.copy()
user_set = get_kfold_users(trn_df, n=k_fold)

score_list = []
score_df = trn_df[['suv', 'itemId', 'label']]
sub_preds = np.zeros(test.shape[0])

dense_fea = [x for x in dense_fea if x != 'label']
x_tst, dnn_feature_columns = get_din_feats_columns(test, dense_fea, 
                                                   sparse_fea, behavior_fea, hist_behavior_fea, max_len=32)

In [7]:
# 五折交叉验证，并将中间结果保存用于staking
for n_fold, valid_user in enumerate(user_set):
    

    
    print("1")
    train_idx = trn_df[~trn_df['suv'].isin(valid_user)] # add slide user
    valid_idx = trn_df[trn_df['suv'].isin(valid_user)]
    
    print("2")
    # 准备训练数据
    x_trn, dnn_feature_columns = get_din_feats_columns(train_idx, dense_fea, 
                                                       sparse_fea, behavior_fea, hist_behavior_fea, max_len=32)
    y_trn = train_idx['label'].values

    print("3")
    # 准备验证数据
    x_val, dnn_feature_columns = get_din_feats_columns(valid_idx, dense_fea, 
                                                   sparse_fea, behavior_fea, hist_behavior_fea, max_len=32)
    y_val = valid_idx['label'].values
    
        # 建立模型
    model = DIN(dnn_feature_columns, behavior_fea)
    # 模型编译
    model.compile('adam', 'binary_crossentropy',metrics=['binary_crossentropy','accuracy'])
    
    print("4")
    history = model.fit(x_trn, y_trn, verbose=1, epochs=3, validation_data=(x_val, y_val) , batch_size=256)
    
    # print("5")
    # # 预测验证集结果
    # valid_idx['rate'] = model.predict(x_val, verbose=1, batch_size=256)   
    
    # print("6")
    # valid_idx.sort_values(by=['suv', 'rate'])
    # valid_idx['rate_new'] = valid_idx.groupby(['suv'])['rate'].rank(ascending=False, method='first')
    
    # print("7")
    # # 将验证集的预测结果放到一个列表中，后面进行拼接
    # score_list.append(valid_idx[['suv', 'itemId', 'rate', 'rate_new']])
    
    print("8")
    # # 如果是线上测试，需要计算每次交叉验证的结果相加，最后求平均
    # if not offline:
    #     sub_preds += model.predict(x_tst, verbose=1, batch_size=256)[:, 0] 
    sub_preds += model.predict(x_tst, verbose=1, batch_size=256)[:, 0] 

1
2
3
4
Train on 3387554 samples, validate on 846250 samples


2022-05-16 22:58:44.646834: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-16 22:58:44.850677: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA RTX A5000 major: 8 minor: 6 memoryClockRate(GHz): 1.695
pciBusID: 0000:41:00.0
2022-05-16 22:58:44.850731: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-05-16 22:58:44.856470: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-05-16 22:58:44.859152: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-05-16 22:58:44.859404: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2022-05-16 22:58:44.859823: I tensorflow/stream_executor/platform/defaul

Epoch 1/3


2022-05-16 22:58:47.008173: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-05-16 22:58:48.354020: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


3387136/3387554 [============================>.] - ETA: 0s - loss: 0.4705 - binary_crossentropy: 0.4668 - acc: 0.8103

...

1422218/1422218 [==============================] - 29s 20us/sample
1
2
3
4
Train on 3386979 samples, validate on 846825 samples
Epoch 1/3
3386112/3386979 [============================>.] - ETA: 0s - loss: 0.4706 - binary_crossentropy: 0.4669 - acc: 0.8103

2022-05-16 23:41:32.185529: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 0, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-05-16 23:41:32.186701: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 1, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-05-16 23:41:32.187951: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] layout failed: Invalid argument: The graph couldn't be sorted in topological order.
2022-05-16 23:41:32.190504: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] model_pruner failed: Invalid argument: MutableGraphView::MutableGraphView error: node 'attention_sequence_pooling_layer_4/local_activation_unit_4/concat' has self cycle fanin 'attention_sequence_pooling_layer_4/local_activation_unit_4/concat'.
2022-05-16 23:41:32.195941: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] remappe

3386979/3386979 [==============================] - 184s 54us/sample - loss: 0.4706 - binary_crossentropy: 0.4669 - acc: 0.8103 - val_loss: 0.4732 - val_binary_crossentropy: 0.4682 - val_acc: 0.8086
Epoch 2/3
3386979/3386979 [==============================] - 191s 56us/sample - loss: 0.4616 - binary_crossentropy: 0.4553 - acc: 0.8136 - val_loss: 0.4725 - val_binary_crossentropy: 0.4661 - val_acc: 0.8089
Epoch 3/3
3386979/3386979 [==============================] - 185s 55us/sample - loss: 0.4565 - binary_crossentropy: 0.4485 - acc: 0.8156 - val_loss: 0.4745 - val_binary_crossentropy: 0.4664 - val_acc: 0.8091
8
   2304/1422218 [..............................] - ETA: 5:15 

2022-05-16 23:48:12.262680: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 0, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-05-16 23:48:12.263580: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 1, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-05-16 23:48:12.264535: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] layout failed: Invalid argument: The graph couldn't be sorted in topological order.
2022-05-16 23:48:12.266125: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] model_pruner failed: Invalid argument: MutableGraphView::MutableGraphView error: node 'attention_sequence_pooling_layer_4/local_activation_unit_4/concat' has self cycle fanin 'attention_sequence_pooling_layer_4/local_activation_unit_4/concat'.
2022-05-16 23:48:12.270627: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] remappe

1422218/1422218 [==============================] - 30s 21us/sample


In [8]:
# 生成提交文件
sub_path='./data/submission/'
sub=pd.read_csv(sub_path+'section2.txt', sep='\t')
sub['result'] = sub_preds/k_fold
sub.to_csv('./data/section2DIN0.txt', sep='\t', index=False)

In [9]:
print(

SyntaxError: unexpected EOF while parsing (149104261.py, line 1)

In [ ]:
# 准备训练数据
x_trn, dnn_feature_columns = get_din_feats_columns(train, dense_fea, sparse_fea, behavior_fea, hist_behavior_fea, max_len=32)
y_trn =train['label'].values

x_tst, dnn_feature_columns = get_din_feats_columns(test, dense_fea, sparse_fea, behavior_fea, hist_behavior_fea, max_len=32)
y_tst =test['label'].values

In [ ]:
history = model.fit(x_trn, y_trn, verbose=1,epochs=3,validation_split=0.2,batch_size=256)

In [ ]:
# 模型预测
test['rate'] = model.predict(x_tst, verbose=1, batch_size=256)
test[['suv', 'itemId', 'rate']].to_csv('./din_rank_score.csv', index=False)

In [ ]:
ygq=list(test['rate'])

# 生成提交文件
sub_path='./data/submission/'
sub=pd.read_csv(sub_path+'section2.txt', sep='\t')
sub['result'] = ygq
sub.to_csv('./data/section2DIN0.txt', sep='\t', index=False)

In [1]:
pip list

Package                        Version
------------------------------ -------------------
absl-py                        1.0.0
anyio                          3.3.4
argon2-cffi                    21.1.0
astor                          0.8.1
astunparse                     1.6.3
attrs                          21.2.0
Babel                          2.9.1
backcall                       0.2.0
bleach                         4.1.0
brotlipy                       0.7.0
certifi                        2021.5.30
cffi                           1.14.6
chardet                        4.0.0
conda                          4.10.3
conda-package-handling         1.7.3
cryptography                   3.4.7
cycler                         0.11.0
debugpy                        1.5.1
decorator                      5.1.0
deepctr                        0.9.0
defusedxml                     0.7.1
entrypoints                    0.3
gast                           0.3.3
gensim                         4.2.0
google-pasta   